# Classification Analysis on Insurance Category on XXX
* Chi Phuong Dao
* I Putu Agastya Harta Pratama

Faculty of Economic Sciences <br>
University of Warsaw <br>
Warsaw, Poland <br>
2025

## Imports

In [30]:
!pip install pandas numpy scikit-learn seaborn statsmodels matplotlib xgboost lightgbm mlxtend

  Using cached statsmodels-0.14.4-cp312-cp312-macosx_11_0_arm64.whl.metadata (9.2 kB)
  Using cached xgboost-3.0.0-py3-none-macosx_12_0_arm64.whl.metadata (2.1 kB)
  Using cached lightgbm-4.6.0-py3-none-macosx_12_0_arm64.whl.metadata (17 kB)
  Using cached mlxtend-0.23.4-py3-none-any.whl.metadata (7.3 kB)
  Using cached patsy-1.0.1-py2.py3-none-any.whl.metadata (3.3 kB)
Using cached statsmodels-0.14.4-cp312-cp312-macosx_11_0_arm64.whl (9.9 MB)
Using cached xgboost-3.0.0-py3-none-macosx_12_0_arm64.whl (2.0 MB)
Using cached lightgbm-4.6.0-py3-none-macosx_12_0_arm64.whl (1.6 MB)
Using cached mlxtend-0.23.4-py3-none-any.whl (1.4 MB)
Using cached patsy-1.0.1-py2.py3-none-any.whl (232 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [mlxtend]m4/5 [mlxtend]els]


In [14]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
import seaborn as sns   
import pickle

In [33]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
from scipy import stats
import sklearn as sk
import itertools
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
from statsmodels.graphics.mosaicplot import mosaic

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn import svm
import xgboost as xgb
import lightgbm as lgb
from mlxtend.classifier import StackingClassifier

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV


In [17]:
train = pd.read_csv("insurance_train.csv")

In [18]:
test = pd.read_csv("insurance_test.csv")

In [20]:
train_no_target = train.drop(columns=['claim_status'])
all_data = pd.concat([train_no_target, test], ignore_index=True)

## Exploratory Data Analysis

In [21]:
train.head()

,reward,claim_status,person_gender,entity_type,channel,support_interactions,agent_id,customer_score,entity_a,person_age,location,revenue,product_id,trip_length
0,0.000000,0,9b2d5b46,type_a,web,0,agt_0004,0.284509,50b3e71e,51,united states,105.459587,SecurePlan Flex,21
1,18.428627,0,9b2d5b46,type_a,web,0,agt_0001,0.207538,96d6c6df,32,spain,30.433459,DriveSafe Rental Addon,6
2,0.000000,0,9b2d5b46,type_a,web,1,agt_0004,0.300461,50b3e71e,38,new zealand,78.728474,TripGuard Cancel,84
3,0.000000,0,9b2d5b46,type_a,web,0,agt_0004,0.132404,50b3e71e,30,thailand,17.146843,TripGuard Cancel,39
4,0.000000,0,9b2d5b46,type_a,web,2,agt_0004,0.070443,50b3e71e,37,viet nam,10.476416,TripGuard Cancel,48


In [22]:
train.describe()

,reward,claim_status,support_interactions,customer_score,person_age,revenue,trip_length
count,56993.000000,56993.000000,56993.000000,56993.000000,56993.000000,56993.000000,56993.000000
mean,9.794976,0.014633,1.093941,0.189540,39.697261,40.750070,49.234888
std,19.791119,0.120081,0.923113,0.080641,12.603433,48.749221,98.659469
min,0.000000,0.000000,0.000000,0.000000,18.000000,-406.642669,1.000000
25%,0.000000,0.000000,0.000000,0.136389,34.000000,18.246389,9.000000
50%,0.000000,0.000000,1.000000,0.181041,36.000000,26.785242,22.000000
75%,11.336612,0.000000,2.000000,0.230836,44.000000,47.809948,53.000000
max,282.073580,1.000000,6.000000,1.000000,100.000000,801.049866,4856.000000


In [25]:
train.isnull().sum()

reward                  0
claim_status            0
person_gender           0
entity_type             0
channel                 0
support_interactions    0
agent_id                0
customer_score          0
entity_a                0
person_age              0
location                0
revenue                 0
product_id              0
trip_length             0
dtype: int64

In [26]:
test.isnull().sum()

reward                  0
person_gender           0
entity_type             0
channel                 0
support_interactions    0
agent_id                0
customer_score          0
entity_a                0
person_age              0
location                0
revenue                 0
product_id              0
trip_length             0
dtype: int64

In [24]:
train.nunique()

reward                  25246
claim_status                2
person_gender               3
entity_type                 2
channel                     2
support_interactions        7
agent_id                   16
customer_score          56862
entity_a                   16
person_age                 74
location                  146
revenue                 55300
product_id                 26
trip_length               448
dtype: int64

There is no missing value for both the training and testing dataset 

In [27]:
claim_counts = train['claim_status'].value_counts().sort_index()
claim_pct    = (claim_counts / len(df) * 100).round(2)

claim_df = (
    pd.DataFrame({'count': claim_counts, 'percentage_%': claim_pct})
      .rename_axis("claim_status")
)
display(claim_df)                    
cat_cols = ['person_gender', 'entity_type', 'channel',
            'support_interactions', 'agent_id', 'entity_a']

rows = []
for col in cat_cols:
    uniques = train[col].unique()
    rows.append({
        "column": col,
        "n_unique": len(uniques),
        "sample_values": ', '.join(map(str, uniques[:10])) + (' …' if len(uniques) > 10 else '')
    })

unique_df = pd.DataFrame(rows)
display(unique_df)

,count,percentage_%
claim_status,,
0,56159,98.54
1,834,1.46


,column,n_unique,sample_values
0,person_gender,3,"9b2d5b46, f67ab10a, 08f27188"
1,entity_type,2,"type_a, type_b"
2,channel,2,"web, retail"
3,support_interactions,7,"0, 1, 2, 3, 4, 5, 6"
4,agent_id,16,"agt_0004, agt_0001, agt_0002, agt_0005, agt_00..."
5,entity_a,16,"50b3e71e, 96d6c6df, 7b5dbb09, 99ede4e4, 1e089e..."


In general, almost insurance claims got rejected, so there is extreme class imbalance with fewer than 2% of them got accepted. Therefore, a naive model that always predicts "0" would already score 98.5% accuracy while doing nothing useful. Hence, for this dataset, the accuracy is a misleading metric, we will focus on the recall/precision, ROC-AUC instead. 